# GOOGLE MAP API

# IMPORT LIBS

In [46]:
import pandas as pd
import json
import requests

# OPEN DATASET

In [47]:
df = pd.read_csv('../../DATA/PREPROCESSING/prepro_01_geo.csv', nrows=1000000)
df

,key,fare_amount,pickup_datetime,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,passenger_count,Distance
0,2009-06-15 17:26:21.0000001,4.5,2009-06-15 17:26:21 UTC,-73.844311,40.721319,-73.841610,40.712278,1,1.029601
1,2011-08-18 00:35:00.00000049,5.7,2011-08-18 00:35:00 UTC,-73.982738,40.761270,-73.991242,40.750562,2,1.389132
2,2012-04-21 04:30:42.0000001,7.7,2012-04-21 04:30:42 UTC,-73.987130,40.733143,-73.991567,40.758092,1,2.795790
3,2010-03-09 07:51:00.000000135,5.3,2010-03-09 07:51:00 UTC,-73.968095,40.768008,-73.956655,40.783762,1,1.998338
4,2011-01-06 09:50:45.0000002,12.1,2011-01-06 09:50:45 UTC,-74.000964,40.731630,-73.972892,40.758233,1,3.787980
...,...,...,...,...,...,...,...,...,...
999995,2009-08-04 22:54:00.00000014,9.7,2009-08-04 22:54:00 UTC,-73.805007,40.681537,-73.805007,40.681537,1,0.000000
999996,2011-07-23 13:31:00.00000038,4.5,2011-07-23 13:31:00 UTC,-73.973657,40.784485,-73.980912,40.779282,1,0.841954
999997,2013-07-15 20:27:40.0000001,8.0,2013-07-15 20:27:40 UTC,-74.003624,40.742007,-73.981124,40.750336,1,2.113427
999998,2013-11-12 08:32:30.0000005,19.5,2013-11-12 08:32:30 UTC,-73.958898,40.780737,-73.996731,40.742683,2,5.297448


In [48]:
def get_keys(path):
    with open(path) as f:
        return json.load(f)
API_key = get_keys("../../ASSETS/GMAP/api_key.json")
google_key = list(API_key.values())[0]

In [ ]:
lat_origin = df['pickup_latitude'].tolist()
long_origin = df['pickup_longitude'].tolist()
lat_destination = df['dropoff_latitude'].tolist()
long_destination = df['dropoff_longitude'].tolist()

distance = []
duration = []
for i in range(len(long_destination)):
    url = f"https://maps.googleapis.com/maps/api/distancematrix/json?units=imperial&origins={lat_origin[i]},{long_origin[i]}&destinations={lat_destination[i]}%2C{long_destination[i]}&key={google_key}"
    r=requests.get(url)
    data = r.json()
    try:
        distance.append(data['rows'][0]['elements'][0]['distance']['text'])
        duration.append(data['rows'][0]['elements'][0]['duration']['text'])
    except:
        pass

In [ ]:
distance_cleaned = []
duration_cleaned = []
for i in range(len(distance)):
    distance_clean = distance[i].replace(' mi', '').replace(' ft', '')
    float_distance_clean = float(distance_clean)
    distance_cleaned.append(float_distance_clean)
    duration_clean = duration[i].replace(' mins', '').replace(' min', '')
    float_duration_clean = float(duration_clean)
    duration_cleaned.append(float_duration_clean)


df['Driving distance (miles)'] = distance_cleaned
df['Driving duration'] = duration_cleaned
df.head()

In [ ]:
df.to_csv('../../DATA/PREPROCESSING/prepro_01a_geo_gmap.csv', index=False)